# Load Data



### Set env variables

assumes that `SOCRATA_APP_TOKEN`, `SOCRATA_API_KEY_ID`, and `SOCRATA_API_KEY_SECRET` are set in `.env`

In [1]:
from dotenv import load_dotenv

load_dotenv()


True

### Load Packages

In [2]:
import os
import sys

INGESTION_PATH = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
sys.path.insert(0, INGESTION_PATH)

from ingestion import fetch


ImportError: cannot import name 'SCHEMA' from 'ingestion.config' (c:\Users\gorav\GitHub\nyc-311-service-requests\src\ingestion\config.py)

### Full Pull of Data
Takes about 48 minutes

In [ ]:
# await fetch.pull_latest_data()

### Incremental Pull of Data

In [ ]:
await fetch.fetch_current_month(save = True)

Fetching current month: 2025-09
Fetching 2025-09 ...


### Function

In [5]:
# def _create_socrata_client(api_endpoint, app_token, username, password):
#     """"
    
#     """
#     client = Socrata(api_endpoint, app_token=app_token, username=username, password=password, timeout = 600)
#     return client


In [6]:
# def _fetch_and_save_month_sync(year: int, month: int):
#     """Runs in a worker thread. Creates its own Socrata client (no sharing)."""
#     # per-thread client
#     # client = Socrata("data.cityofnewyork.us", APP_TOKEN)
#     client = _create_socrata_client(api_endpoint="data.cityofnewyork.us", app_token=APP_TOKEN, username=API_KEY_ID, password=API_KEY_SECRET)

#     start = f"{year}-{month:02d}-01T00:00:00"
#     if month == 12:
#         end = f"{year+1}-01-01T00:00:00"
#     else:
#         end = f"{year}-{month+1:02d}-01T00:00:00"

#     where_clause = f"{date_column} >= '{start}' AND {date_column} < '{end}'"
#     print(f"Fetching {year}-{month:02d} ...")

#     # client.get_all handles paging internally (blocking)
#     results = list(client.get_all(dataset_id, where=where_clause))

#     if results:
#         df = pd.DataFrame.from_records(results)
#         # write to year/month partition
#         file_path = os.path.join(output_dir, f"year={year}/month={month:02d}/part-0000.parquet")
#         os.makedirs(os.path.dirname(file_path), exist_ok=True)
#         # choose your engine; fastparquet avoids some pyarrow quirks in notebooks
#         df.to_parquet(file_path, index=False, engine="fastparquet")
#         print(f"Saved {file_path} ({len(df):,} rows)")

#         # clean up memory
#         del df
#         del results
#         gc.collect()
#     else:
#         print(f"No data for {year}-{month:02d}")

# async def fetch_and_save(year: int, month: int, sem: asyncio.Semaphore):
#     async with sem:
#         # run the sync worker in a thread
#         await asyncio.to_thread(_fetch_and_save_month_sync, year, month)

# async def pull_latest_data():
#     sem = asyncio.Semaphore(MAX_CONCURRENCY)
#     tasks = [fetch_and_save(y, m, sem) for y in years for m in months]
#     await asyncio.gather(*tasks)


### Apply Function

In [7]:
# dataset_id = "erm2-nwe9"
# date_column = "created_date"
# output_dir = "../data/311-service-requests"
# os.makedirs(output_dir, exist_ok=True)

# years = range(2010, 2026)   # e.g., 2010–2011
# months = range(1, 13)       # 1..12
# MAX_CONCURRENCY = 20         # tune for your machine / API limits

In [8]:
# await pull_latest_data()

### Load Data

In [9]:
import polars as pl
import numpy as np
from pathlib import Path


## Exploratory Data Analysis with Polars


### Load all parquet files using Polars lazy API


### Basic Information


In [24]:
# Get shape and basic info
print("Dataset Shape:")
print(f"Rows: {lf.select(pl.len()).collect().item():,}")
print(f"Columns: {len(lf.collect_schema())}")
print(f"\nColumn Names and Types:")
for col, dtype in lf.collect_schema().items():
    print(f"  {col}: {dtype}")


Dataset Shape:
Rows: 41,178,052
Columns: 43

Column Names and Types:
  unique_key: String
  created_date: Datetime(time_unit='ms', time_zone=None)
  closed_date: Datetime(time_unit='ms', time_zone=None)
  agency: String
  agency_name: String
  complaint_type: String
  descriptor: String
  location_type: String
  incident_zip: String
  incident_address: String
  street_name: String
  cross_street_1: String
  cross_street_2: String
  intersection_street_1: String
  intersection_street_2: String
  address_type: String
  city: String
  landmark: String
  facility_type: String
  status: String
  due_date: Datetime(time_unit='ms', time_zone=None)
  resolution_description: String
  resolution_action_updated_date: Datetime(time_unit='ms', time_zone=None)
  community_board: String
  bbl: String
  borough: String
  x_coordinate_state_plane: Float64
  y_coordinate_state_plane: Float64
  open_data_channel_type: String
  park_facility_name: String
  park_borough: String
  vehicle_type: String
  tax

### Preview the Data


In [25]:
# Collect a sample to view
lf.head(10).collect()


SchemaError: data type mismatch for column created_date: incoming: String != target: Datetime('ms')

### Missing Values Analysis


In [11]:
df.collect_schema()

Schema([('unique_key', String),
        ('created_date', String),
        ('closed_date', String),
        ('agency', String),
        ('agency_name', String),
        ('complaint_type', String),
        ('descriptor', String),
        ('location_type', String),
        ('incident_zip', String),
        ('incident_address', String),
        ('street_name', String),
        ('cross_street_1', String),
        ('cross_street_2', String),
        ('address_type', String),
        ('city', String),
        ('facility_type', String),
        ('status', String),
        ('resolution_description', String),
        ('resolution_action_updated_date', String),
        ('community_board', String),
        ('bbl', String),
        ('borough', String),
        ('x_coordinate_state_plane', String),
        ('y_coordinate_state_plane', String),
        ('open_data_channel_type', String),
        ('park_facility_name', String),
        ('park_borough', String),
        ('latitude', String),
        ('

In [28]:


df.select(pl.col('vehicle_type').null_count()).collect()

ColumnNotFoundError: did not find column vehicle_type, consider passing `missing_columns='insert'`

In [ ]:
# Calculate the number of nulls in the 'vehicle_type' column
df.select(pl.col('vehicle_type').is_null().sum().alias('vehicle_type_nulls')).collect()

ColumnNotFoundError: did not find column vehicle_type, consider passing `missing_columns='insert'`

In [10]:
# Calculate null counts and percentages
total_rows = df.select(pl.len()).collect().item()

null_counts = df.select([
    pl.col(col).is_null().sum().alias(col) 
    for col in df.collect_schema().names()
]).collect()

# Convert to long format for better readability
null_summary = pl.DataFrame({
    "Column": null_counts.columns,
    "Null_Count": null_counts.row(0),
}).with_columns([
    (pl.col("Null_Count") / total_rows * 100).round(2).alias("Null_Percentage")
]).sort("Null_Count", descending=True)

print(f"Missing Values Summary (Total Rows: {total_rows:,})")
null_summary.head(20)


ColumnNotFoundError: did not find column vehicle_type, consider passing `missing_columns='insert'`

### Top Complaint Types


In [ ]:
# Top complaint types
top_complaints = df.group_by("complaint_type").agg([
    pl.len().alias("count")
]).sort("count", descending=True).limit(20).collect()

print("Top 20 Complaint Types:")
top_complaints


### Temporal Analysis


In [ ]:
# Requests over time (by year-month)
temporal = df.with_columns([
    pl.col("created_date").str.to_datetime().alias("created_dt")
]).with_columns([
    pl.col("created_dt").dt.year().alias("year"),
    pl.col("created_dt").dt.month().alias("month")
]).group_by(["year", "month"]).agg([
    pl.len().alias("num_requests")
]).sort(["year", "month"]).collect()

print("Service Requests Over Time:")
temporal


### Geographic Distribution


In [ ]:
# Requests by borough
by_borough = df.group_by("borough").agg([
    pl.len().alias("count")
]).sort("count", descending=True).collect()

print("Service Requests by Borough:")
by_borough


### Agency Analysis


In [ ]:
# Top agencies handling requests
top_agencies = df.group_by("agency").agg([
    pl.len().alias("count")
]).sort("count", descending=True).limit(15).collect()

print("Top 15 Agencies:")
top_agencies


### Status Distribution


In [ ]:
# Request status distribution
status_dist = df.group_by("status").agg([
    pl.len().alias("count")
]).sort("count", descending=True).collect()

print("Status Distribution:")
status_dist


### Response Time Analysis


In [ ]:
# Calculate response times where both created_date and closed_date exist
response_times = df.filter(
    pl.col("closed_date").is_not_null() & pl.col("created_date").is_not_null()
).with_columns([
    pl.col("created_date").str.to_datetime().alias("created_dt"),
    pl.col("closed_date").str.to_datetime().alias("closed_dt")
]).with_columns([
    (pl.col("closed_dt") - pl.col("created_dt")).dt.total_hours().alias("response_hours")
]).filter(
    pl.col("response_hours") >= 0  # Only positive response times
).select([
    pl.col("response_hours").mean().alias("mean_hours"),
    pl.col("response_hours").median().alias("median_hours"),
    pl.col("response_hours").std().alias("std_hours"),
    pl.col("response_hours").min().alias("min_hours"),
    pl.col("response_hours").max().alias("max_hours"),
    pl.col("response_hours").quantile(0.25).alias("q25_hours"),
    pl.col("response_hours").quantile(0.75).alias("q75_hours"),
    pl.col("response_hours").quantile(0.90).alias("q90_hours"),
    pl.len().alias("count_with_close_date")
]).collect()

print("Response Time Statistics (in hours):")
response_times


### Year-over-Year Growth


In [ ]:
# Yearly trends
yearly = df.with_columns([
    pl.col("created_date").str.to_datetime().dt.year().alias("year")
]).group_by("year").agg([
    pl.len().alias("num_requests")
]).sort("year").collect()

print("Year-over-Year Requests:")
yearly


### Top Complaint Types by Borough


In [ ]:
# Top complaints by borough
complaints_by_borough = df.group_by(["borough", "complaint_type"]).agg([
    pl.len().alias("count")
]).sort(["borough", "count"], descending=[False, True]).collect()

# Get top 5 complaints per borough
top_per_borough = complaints_by_borough.group_by("borough").head(5)

print("Top 5 Complaint Types per Borough:")
top_per_borough
